# Hypothesis Testing

In this project I test the following hypothesis: university towns in the US have their mean housing prices less effected by recessions. 

Let m1 = PriceRatioUni = (mean housing prices in university towns the quarter before recession)/(mean housing prices in university towns at recession bottom).
Let m2 = PriceRatioUS = (mean housing prices in the US the quarter before recession)/(mean housing prices in the US at recession bottom).

Null hypothesis: m1 $\geq$ m2

Alternative hypothesis: m1 $<$ m2

This project is based on one of the assignments in the course "Introduction to Data Science in Python" on Coursera provided by University of Michigan.

Definitions:
- A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
- A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
- A _recession bottom_ is the quarter within a recession which had the lowest GDP.
- A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

The following data files are used:
- From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
- From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
- From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this project, I only look at GDP data from the first quarter of 2000 onward.

In [5]:
import pandas as pd
import numpy as np
from scipy import stats

In [6]:
# dictionary which maps two letter state acronyms to state names
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [7]:
def get_list_of_university_towns():
    
    university_towns=[]
    with open('university_towns.txt') as file:
        for line in file:
            if 'edit' in line:
                state = line
            else:
                university_towns.append((state, line))
    towns_df2 = pd.DataFrame(university_towns, columns=['State', 'RegionName'])

    def get_citystate(item):
        if '(' in item:
            return item[:item.find(' (')]
        elif '[' in item:
            return item[:item.find('[')]
        elif ':\n' in item:
            return item[:item.find('\n')]
        elif '\n' in item:
            return item[:item.find('\n')]
        else:
            return item
    towns_df = towns_df2.applymap(get_citystate)
    

    '''Returns a DataFrame of towns and the states they are in from the
    university_towns.txt list. The format of the DataFrame is:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ],
    columns=["State", "Region Name"]  )

    The following cleaning is done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Removing newline characters '\n'. '''

    return towns_df

#pd.set_option('display.max_rows', 1000)

get_list_of_university_towns().head(10)

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [8]:
def get_recession_start():
    GDP = pd.read_excel('gdplev.xls', usecols=[4,6], skiprows=[i for i in range(220)], header=None)
    GDP.rename(columns={4: 'Year, Q', 6: 'GDP'}, inplace=True)
    for i in range(len(GDP)-2):
        if GDP['GDP'].iloc[i+1] < GDP['GDP'].iloc[i] and GDP['GDP'].iloc[i+2] < GDP['GDP'].iloc[i+1]:
            r_start = GDP['Year, Q'].iloc[i+1]
            break
    '''Returns the year and quarter of the recession start time as a
    string value in a format such as 2005q3'''
    #n = GDP[GDP['Year, Q'] == r_start].index.values
    return r_start # GDP['Year, Q'].iloc[n-1]


get_recession_start()

'2008q3'

In [9]:
def get_recession_end():
    GDP = pd.read_excel('gdplev.xls', usecols=[4,6], skiprows=[i for i in range(220)], header=None)
    GDP.rename(columns={4: 'Year, Q', 6: 'GDP'}, inplace=True)
    for i in range(len(GDP[GDP['Year, Q']>='2008q2']), len(GDP) -2):
        if GDP['GDP'].iloc[i+1] > GDP['GDP'].iloc[i] and GDP['GDP'].iloc[i+2] > GDP['GDP'].iloc[i+1]:
            r_end = GDP['Year, Q'].iloc[i+2]
            break
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    return r_end

get_recession_end()

'2009q4'

In [10]:
def get_recession_bottom():
    GDP = pd.read_excel('gdplev.xls', usecols=[4,6], skiprows=[i for i in range(220)], header=None)
    GDP.rename(columns={4: 'Year, Q', 6: 'GDP'}, inplace=True)
    recession_df = GDP[(GDP['Year, Q']>='2008q2') & (GDP['Year, Q']<='2009q2')]        
    index_min = recession_df['GDP'].idxmin()
    r_bottom = recession_df['Year, Q'].loc[index_min]
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    return r_bottom

get_recession_bottom()

'2009q2'

In [11]:
def convert_housing_data_to_quarters():
    h_df = pd.read_csv('City_Zhvi_AllHomes.csv', usecols=np.r_[1:3, 51:251])
    h_df.rename(columns={'RegionName': 'Region Name'}, inplace=True)
    #h_df = h_df.fillna(0.0)
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National',
              'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana',
              'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho',
              'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan',
              'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico',
              'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa',
              'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana',
              'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California',
              'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island',
              'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia',
              'ND': 'North Dakota', 'VA': 'Virginia'}
    h_df.replace({'State': states}, inplace=True)
    h_df.set_index(['State', 'Region Name'], inplace=True)
    h_df.rename(columns={'RegionName': 'Region Name'}, inplace=True)
    
    h_df.columns = pd.to_datetime(h_df.columns)
    h_df = h_df.resample('Q', axis=1).mean()
    
    h_df = h_df.rename(columns = lambda x: str(x.to_period('Q')).lower())
    
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This is a dataframe with columns for 2000q1 
    through 2016q3 with the multi-index in the shape of 
    ["State","RegionName"].'''
    
    return h_df 

convert_housing_data_to_quarters().head()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,Region Name,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0


In [15]:
def run_ttest():
    GDP = pd.read_excel('gdplev.xls', usecols=[4,6], skiprows=[i for i in range(220)], header=None)
    GDP.rename(columns={4: 'Year, Q', 6: 'GDP'}, inplace=True)
    h_df = convert_housing_data_to_quarters()
    r_start=get_recession_start()
    r_bottom = get_recession_bottom()
    
    towns_df = get_list_of_university_towns()
    '''states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National',
              'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana',
              'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho',
              'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan',
              'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico',
              'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa',
              'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana',
              'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California',
              'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island',
              'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia',
              'ND': 'North Dakota', 'VA': 'Virginia'}
    states_rev = dict((v,k) for k, v in states.items())
    towns_df.replace({'State': states_rev}, inplace=True)'''

    n = GDP[GDP['Year, Q'] == r_start].index.values     # n[0]=33 - index of '2008q2'
    qrt_bfr_r_start = GDP['Year, Q'].loc[n-1].item()    # '2008q1'
    h_df['PriceRatio'] = h_df[qrt_bfr_r_start].div(h_df[r_bottom])

    #towns_list = towns_df.to_records(index=False).tolist()
    #towns_list2 = pd.MultiIndex.from_tuples(towns_list, names=('State', 'RegionName'))
    #group1 = h_df.loc[towns_list]
    #group1 = h_df.loc[h_df.index.tolist() == towns_list]
    #group2

    df = pd.merge(h_df.reset_index(), towns_df, how='outer', left_on=['State', 'Region Name'], right_on=['State', 'RegionName'], indicator='_flag') # on=towns_df.columns.tolist(),
    group1 = df[df['_flag']=='both']
    group2 = df[df['_flag']!='both']

    t_test = stats.ttest_ind(group1['PriceRatio'], group2['PriceRatio'], nan_policy='omit')
    t_statistic = t_test[0]
    p = t_test[1]
    different = p < 0.01

    m1=group1['PriceRatio'].mean()    # uni
    m2=group2['PriceRatio'].mean()    # non-uni
    if m1 < m2:
        better = 'university town'
    else:
        better = 'non-university town'


    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a t-test
    comparing the university town values to the non-university towns values,
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence.

    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (reject the null hypothesis), or different=False if
    otherwise (cannot reject the null hypothesis).'''

    return different, p, better

run_ttest()


(True, 0.002724063704753125, 'university town')

## Conclusion

There is enough evidence to reject the null hypothesis. So, university towns in the US have their mean housing prices less effected by recessions.